### Import libraries

In [3]:
import os
import pandas as pd
import numpy as np



#set working directory to this from the workspace
os.chdir('/Users/janjarco/Programming/PrivateRepository/FlightDataThesisProject')

#list all the files in current working directory
print(os.listdir("data"))


['.DS_Store', 'external', 'interim', 'processed', 'raw']


#### Reading the merged data frame of orders and clicks

In [4]:
import time
start_time = time.time()
df = pd.read_csv("data/processed/clicks_orders_merge_filtered_currency.csv")
end_time = time.time()
elapsed_time = end_time - start_time
print('Time elapsed (in seconds):', elapsed_time)

/var/folders/gp/cz1c1p2d6ln8dw3vyy038cwm0000gn/T/ipykernel_89154/192662082.py:3: DtypeWarning: Columns (83,116,123,124,126,129,130,131,132,133,156,157) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/processed/clicks_orders_merge_filtered_currency.csv")


Time elapsed (in seconds): 2.453141927719116


In [14]:
# filter df.columns.values  by regex
def filter_columns_by_regex(df, regex):
    import re
    return [col for col in df.columns.values if bool(re.search(pattern=str(regex), string=str(col)))]

In [15]:
filter_columns_by_regex(df, 'experiments')

['orders_search_data.experiments.202206-direct-to-step2',
 'orders_search_data.experiments.202212-new-sorting',
 'orders_search_data.experiments.202301-conditions-text',
 'orders_search_data.experiments.202204-danish-explanations',
 'orders_search_data.experiments.202206-sabre-gateway-migration',
 'orders_search_data.experiments.202202-test-experiment',
 'orders_search_data.experiments.202202-handbag-ordering',
 'orders_search_data.experiments.202203-german-background-image',
 'orders_search_data.experiments.202203-mix-classic',
 'orders_search_data.experiments.202205-meta-per-pax']

In [45]:
# summarize df by orders_created_at_date and caluculate counts 
df.groupby(["orders_created_at_date"])['clicks_index'].count()


orders_created_at_date
2022-01-01     96
2022-01-02    154
2022-01-03    149
2022-01-04    149
2022-01-05    145
             ... 
2023-03-16    352
2023-03-17    329
2023-03-18    326
2023-03-19    482
2023-03-20    223
Name: clicks_index, Length: 444, dtype: int64

In [23]:
# calculate total price in SEK
df['orders_total_price_SEK'] = df['clicks_itinerary_sales_price'] * df['exchange_rate']
# df['orders_total_price_SEK'].hist(bins=100)

# histogram of clicks_itinerary_travel_time in hours
df['clicks_itinerary_travel_timehours'] = round(df['clicks_itinerary_travel_time'] / 60,2)
# df['clicks_itinerary_travel_timehours'].hist(bins=100)

In [20]:
experiment_vars = filter_columns_by_regex(df, '.*experiments')
experiment_vars

['orders_search_data.experiments.202206-direct-to-step2',
 'orders_search_data.experiments.202212-new-sorting',
 'orders_search_data.experiments.202301-conditions-text',
 'orders_search_data.experiments.202204-danish-explanations',
 'orders_search_data.experiments.202206-sabre-gateway-migration',
 'orders_search_data.experiments.202202-test-experiment',
 'orders_search_data.experiments.202202-handbag-ordering',
 'orders_search_data.experiments.202203-german-background-image',
 'orders_search_data.experiments.202203-mix-classic',
 'orders_search_data.experiments.202205-meta-per-pax']

In [21]:
from scipy import stats as stats

def perform_t_test(dataframe, group_var, var_test, group1, group2):
    print("test performed for "+group_var)
    # Define the two groups to compare
    group1_data = dataframe.loc[dataframe[group_var] == group1, var_test]
    group2_data = dataframe.loc[dataframe[group_var] == group2, var_test]

    # Perform a two-sample t-test assuming unequal variances
    t_stat, p_value = stats.ttest_ind(group1_data, group2_data, equal_var=False)

    # Calculate the mean and count for each group
    group1_mean = group1_data.mean()
    group1_count = group1_data.count()
    group2_mean = group2_data.mean()
    group2_count = group2_data.count()

    # Display the results
    print("Group 1 mean:", group1_mean, "count:", group1_count)
    print("Group 2 mean:", group2_mean, "count:", group2_count)
    
    # print("t-statistic:", t_stat)
    print("p-value:", p_value)



In [24]:

for var in experiment_vars:
    perform_t_test(
    dataframe=df, 
    group_var=var, 
    var_test = 'orders_total_price_SEK', 
    group1='control', 
    group2 = 'hypothesis')

test performed for orders_search_data.experiments.202206-direct-to-step2
Group 1 mean: 5264.466991171823 count: 100854
Group 2 mean: nan count: 0
p-value: nan
test performed for orders_search_data.experiments.202212-new-sorting
Group 1 mean: 4894.973321161475 count: 1926
Group 2 mean: 5120.744760110653 count: 34296
p-value: 0.19021862110009982
test performed for orders_search_data.experiments.202301-conditions-text
Group 1 mean: 6469.570812338355 count: 4448
Group 2 mean: 6439.887532140018 count: 4428
p-value: 0.888474767522465
test performed for orders_search_data.experiments.202204-danish-explanations
Group 1 mean: 5234.50480009638 count: 6298
Group 2 mean: 5334.827107774125 count: 6083
p-value: 0.3517088608483724
test performed for orders_search_data.experiments.202206-sabre-gateway-migration
Group 1 mean: 6339.656490773792 count: 1450
Group 2 mean: 6257.456313094394 count: 1409
p-value: 0.8116251412140219
test performed for orders_search_data.experiments.202202-test-experiment
Grou

In [29]:
addon_vars = filter_columns_by_regex(df, 'orders_addon')
# from a list of variables addon_vars create a sum of all the variables
df['orders_addon_totalsum'] = df[addon_vars].sum(axis=1)


In [30]:
for var in experiment_vars:
    perform_t_test(
    dataframe=df, 
    group_var=var, 
    var_test = 'orders_addon_totalsum', 
    group1='control', 
    group2 = 'hypothesis')

test performed for orders_search_data.experiments.202206-direct-to-step2
Group 1 mean: 0.4750233010093799 count: 100854
Group 2 mean: nan count: 0
p-value: nan
test performed for orders_search_data.experiments.202212-new-sorting
Group 1 mean: 0.48286604361370716 count: 1926
Group 2 mean: 0.46745976207137857 count: 34296
p-value: 0.3656469765328353
test performed for orders_search_data.experiments.202301-conditions-text
Group 1 mean: 0.4660521582733813 count: 4448
Group 2 mean: 0.480126467931346 count: 4428
p-value: 0.3623097299108611
test performed for orders_search_data.experiments.202204-danish-explanations
Group 1 mean: 0.4785646236900603 count: 6298
Group 2 mean: 0.4757520960052606 count: 6083
p-value: 0.8347089132184946
test performed for orders_search_data.experiments.202206-sabre-gateway-migration
Group 1 mean: 0.44551724137931037 count: 1450
Group 2 mean: 0.43789921930447123 count: 1409
p-value: 0.7838634479036116
test performed for orders_search_data.experiments.202202-test-ex

In [35]:
from datetime import datetime

search_engine_changes = {
    'fix.reordering.after.maximum.results': datetime(2023, 1, 5, 11, 50),
    'remove.itineraries.departure.too.close': datetime(2022, 9, 28, 11, 9),
    'lhg.gateway.migration': datetime(2022, 7, 25, 11, 22),
    'add.support.unacceptable.connections': datetime(2023, 2, 16, 12, 55),
    'support.20ITINS': datetime(2022, 12, 21, 16, 23),
    'avoid.cabin.downgrade': datetime(2022, 8, 19, 9, 14),
    'bump.BFM.version': datetime(2022, 8, 12, 9, 29),
    'gateway.migration': datetime(2022, 6, 13, 13, 44),
    'hack.bagprice.override.Altea.FLX': datetime(2022, 9, 15, 7, 50),
    'mixed.itineraries': datetime(2022, 8, 10, 11, 54),
    'add.price.override': datetime(2022, 8, 10, 7, 23),
    'mobile.pay.support.Denmark': datetime(2023, 2, 21, 11, 31),
    'immediate.payment.if.less.36.hours.departure': datetime(2022, 9, 26, 13, 38)
}



In [40]:
# transform the 'clicks_created_at' column to datetime format
df['clicks_created_at'] = pd.to_datetime(df['clicks_created_at'])


ValueError: Tz-aware datetime.datetime cannot be converted to datetime64 unless utc=True

In [41]:

# print the resulting dataframe
df['clicks_created_at']

0         2022-01-01 02:36:08+01:00
1         2022-01-01 06:46:22+01:00
2         2022-01-01 07:35:47+01:00
3         2022-01-01 07:44:40+01:00
4         2022-01-01 08:24:28+01:00
                    ...            
135990    2023-03-20 16:47:05+01:00
135991    2023-03-20 16:53:32+01:00
135992    2023-03-20 17:01:10+01:00
135993    2023-03-20 16:55:17+01:00
135994    2023-03-20 17:03:25+01:00
Name: clicks_created_at, Length: 135995, dtype: object

In [45]:
from datetime import timedelta

def before_after_change(clicks_created_at, change_at):
    if change_at - timedelta(weeks=1) <= clicks_created_at <= change_at + timedelta(weeks=1):
        return 'before_change' if clicks_created_at <= change_at else 'after_change'
    else:
        return None

# create a new column 'before_after_change' and apply the function before_after_change
# df['search_changes_fix.reordering.after.maximum.results'] = 
df.apply(lambda row: before_after_change(row['clicks_created_at'], search_engine_changes['fix.reordering.after.maximum.results']), axis=1)


TypeError: can't compare offset-naive and offset-aware datetimes

In [43]:
search_engine_changes['fix.reordering.after.maximum.results']

datetime.datetime(2023, 1, 5, 11, 50)